In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
from datetime import datetime

In [3]:
cd F:\IwB\semestr1\ziwo

F:\IwB\semestr1\ziwo


In [4]:
# Wczytanie danych
orders = pd.read_csv('orders.csv', sep=";", encoding="ISO-8859-1")
orders1 = pd.read_csv('orders.csv', sep=";", encoding="ISO-8859-1")
orders2 = pd.read_csv('orders.csv', sep=";", encoding="ISO-8859-1")
orderLines = pd.read_csv('orderlines.csv', sep=";")
orderLines1 = pd.read_csv('orderlines.csv', sep=";")
orderLines2 = pd.read_csv('orderlines.csv', sep=";")
products = pd.read_csv('products.csv', sep=";")
products1 = pd.read_csv('products.csv', sep=";")[['productid', 'productgroupname']]
products2 = pd.read_csv('products.csv', sep=";")[['productid', 'productgroupname']]
customers = pd.read_csv('customers.csv', sep=";")
customers1 = pd.read_csv('customers.csv', sep=";")
customers2 = pd.read_csv('customers.csv', sep=";")


In [5]:
orderLinesCJ = orderLines1.merge(orderLines2, on="orderid", suffixes=('_x', '_y')) 
ordersCJ = orders1.merge(orders2, on="orderid", suffixes=('_x', '_y'))
productsCJ = products1.merge(products2, on="productid", suffixes=('_x', '_y'))
customersCJ = customers1.merge(customers2, on="householdid", suffixes=('_x', '_y'))
customersCJ.head(n=2)

,customerid_x,householdid,gender_x,firstname_x,customerid_y,gender_y,firstname_y
0,174596,53949999,M,DANIEL,174596,M,DANIEL
1,68239,49927024,M,JIM,68239,M,JIM


In [7]:
products1["productid_x"] = products1.productid
products2["productid_y"] = products2.productid

In [8]:
mergetables = orderLinesCJ.merge(ordersCJ, on="orderid")
mergetables = mergetables.merge(products1, on ="productid_x")
mergetables = mergetables.merge(products2, on ="productid_y")
mergetables.head(n=2)

,orderlineid_x,orderid,productid_x,shipdate_x,billdate_x,unitprice_x,numunits_x_x,totalprice_x_x,orderlineid_y,productid_y,...,state_y,zipcode_y,paymenttype_y,totalprice_y_y,numorderlines_y,numunits_y_y,productid_x,productgroupname_x,productid_y,productgroupname_y
0,1010561,1006414,10834,2011-03-07 00:00,2011-03-08 00:00,18.0,1,18.0,1010561,10834,...,AL,35222,MC,36.0,2,2,10834,BOOK,10834,BOOK
1,1010589,1008588,10834,2011-03-08 00:00,2011-03-09 00:00,18.0,1,18.0,1010589,10834,...,NY,10021,AE,88.0,5,5,10834,BOOK,10834,BOOK


In [9]:
datab = mergetables[['productid_x','productid_y','orderid','productgroupname_x','productgroupname_y','customerid_x','customerid_y']]
datab.head(n=2)

,productid_x,productid_x,productid_y,productid_y,orderid,productgroupname_x,productgroupname_y,customerid_x,customerid_y
0,10834,10834,10834,10834,1006414,BOOK,BOOK,48167,48167
1,10834,10834,10834,10834,1008588,BOOK,BOOK,107390,107390


In [11]:
#Usunięcie zdublowanych kolumn, powstałych podczas mergowania
datab = datab.loc[:,~datab.columns.duplicated()]
datab.head(n=2)

,productid_x,productid_y,orderid,productgroupname_x,productgroupname_y,customerid_x,customerid_y
0,10834,10834,1006414,BOOK,BOOK,48167,48167
1,10834,10834,1008588,BOOK,BOOK,107390,107390


In [13]:
differentProducts = datab[datab.productid_x < datab.productid_y]
differentProducts.head(n=2)

,productid_x,productid_y,orderid,productgroupname_x,productgroupname_y,customerid_x,customerid_y
3883,10006,10834,1006256,BOOK,BOOK,109455,109455
3884,10006,10834,1006981,BOOK,BOOK,1492,1492


In [15]:
customers1["customerid_x"] = customers1.customerid
customers2["customerid_y"] = customers2.customerid

In [16]:
mergetables2 = differentProducts.merge(customers1, on ="customerid_x")

In [90]:
mergetables2.head(n=2)

,productid_x,productid_y,orderid,productgroupname_x,productgroupname_y,customerid_x,customerid_y,customerid,householdid,gender,firstname
0,10006,10834,1006256,BOOK,BOOK,109455,109455,109455,22103622,M,HARRY
1,10834,10942,1006256,BOOK,BOOK,109455,109455,109455,22103622,M,HARRY


In [111]:
expectedtab = mergetables2[['productid_x','productid_y','orderid','productgroupname_x','productgroupname_y','householdid','orderid']]
expectedtab = expectedtab.groupby(['productid_x','productid_y', 'householdid','productgroupname_x','productgroupname_y']).size().reset_index(name='count')
expectedtab.head(n=10)
expectedtab = expectedtab[expectedtab['count'] > 1]

expectedtabfin = expectedtab.sort_values(by = "count", ascending=False)
expectedtabfin.head(n=10)

,productid_x,productid_y,householdid,productgroupname_x,productgroupname_y,count
98047,11209,12869,23107044,ARTWORK,ARTWORK,30
100599,11717,11746,50520790,ARTWORK,ARTWORK,30
100698,11764,12292,48765412,ARTWORK,ARTWORK,30
100676,11746,12988,50520790,ARTWORK,ARTWORK,30
100781,11796,12082,51035891,ARTWORK,ARTWORK,25
148941,12820,12851,45288212,FREEBIE,OCCASION,25
160803,13039,13056,48925191,ARTWORK,ARTWORK,25
98687,11317,11796,51035891,ARTWORK,ARTWORK,25
100605,11717,12988,50520790,ARTWORK,ARTWORK,25
98689,11317,12082,51035891,ARTWORK,ARTWORK,25
